# Multiscale Migration Model

This notebook implements our model using `numpy` and `pandas` (with `xlrd`). It has been tested to run on Python 3.6. To start, import the required libraries.

In [42]:
import numpy as np
import pandas as pd

There are a number of datasets used in this model. They can all be found in the `/data` subdirectory.

In [43]:
%ls data

'A&B values for RTS.xlsx'
 Country_List_ISO_3166_Codes_Latitude_Longitude.csv
 Freedom_index.xlsx
 other.csv
 PassportIndex.xlsx
 UN_MigrantStockByOriginAndDestination_2015.xlsx
 wb-codes.csv
 wb-original.csv


The following shortcut functions helps locate these data files easily.

In [45]:
def file_path(name):
    """
    Shortcut function to get the relative path to the directory
    which contains the data.
    """
    return "./data/%s" % name

These functions will be useful later when pulling in the data.

In [51]:
def country_codes():
    """
    Build country rows from their names, ISO codes, and Numeric
    Country Codes.
    """
    return (
        pd.read_csv(
            file_path(
                "Country_List_ISO_3166_Codes_Latitude_Longitude.csv"),
            usecols=[0, 2, 3],
            index_col=1,
            keep_default_na=False))

def freedom_index():
    """
    Read data from the Freedom Index.
    """
    # TODO: Add xlrd to requirements.
    xl = pd.ExcelFile(file_path("Freedom_index.xlsx"))
    return xl.parse(1)

def ab_values():
    """
    Read generated A/B values for each country.
    """
    return pd.read_excel(file_path("A&B values for RTS.xlsx")).T

def passport_index():
    """
    Read data from the Passport Index.
    """
    return pd.read_excel(file_path("PassportIndex.xlsx"))

Modify this depending on how many table rows you want to see.

In [48]:
pd.set_option("display.max_rows", 10)

Now that everything is set up, it is time to start processing the data.

### A/B Values

In [49]:
wb_codes = pd.read_csv(file_path("wb-codes.csv"), index_col=0)
ab = ab_values()
ab.index = [wb_codes[wb_codes.index == x]["ISO3"][0] for x in ab_values().index]
ab

,A,B
BDI,72.993,0.0228
NER,113.170,0.0200
MWI,58.581,0.0226
LBR,144.660,0.0197
MOZ,74.334,0.0267
...,...,...
IRL,20473.000,0.0191
NOR,28548.000,0.0161
USA,12001.000,0.0260
CHE,25711.000,0.0193


### Freedom Index

In [52]:
#[cc[cc['Country'] == x]["Alpha-3 code"] if len(cc[cc['Country'] == x]) > 0 else x for x in fi['Country']]
codes = []
other_codes = pd.read_csv(file_path("other.csv"), index_col=0)
cc = country_codes()
fi = freedom_index()
for country in fi['Country']:
    if len(cc[cc['Country'] == country]):
        codes.append(cc[cc['Country'] == country].index[0])
    elif len(other_codes[other_codes.index == country]):
        codes.append(other_codes[other_codes.index == country]["ISO"][0])
    else:
        print(country)
fi['Country'] = codes
fi

,Country,Total Aggr
0,AFG,24
1,ALB,68
2,DZA,35
3,AND,95
4,AGO,24
...,...,...
190,VEN,30
191,VNM,20
192,YEM,14
193,ZMB,56


### Passport Index

In [53]:
pi = passport_index()
#[cc[cc['Country'] == x]["Alpha-3 code"] if len(cc[cc['Country'] == x]) > 0 else x for x in fi['Country']]
codes = []
other_codes = pd.read_csv(file_path("other.csv"), index_col=0)
for country in pi['Country']:
    if len(cc[cc['Country'] == country]):
        codes.append(cc[cc['Country'] == country].index[0])
    elif len(other_codes[other_codes.index == country]):
        codes.append(other_codes[other_codes.index == country]["ISO"][0])
    else:
        print(country)
pi['Country'] = codes
pi

,Country,Rank (1 = most welcoming)
0,KHM,1
1,COM,1
2,CIV,1
3,GNB,1
4,MDG,1
...,...,...
194,AFG,98
195,PRK,98
196,SOM,98
197,SYR,98


In [6]:
passport_index = pi.set_index("Country")
freedom_index = fi.set_index("Country")
#pd.concat([ab, passport_index, freedom_index])
data = ab.join(passport_index).join(freedom_index).dropna()

In [7]:
political_barriers = 2/(1/data['Total Aggr']/sum(1/data['Total Aggr']) +
 data['Rank (1 = most welcoming)']/sum(data['Rank (1 = most welcoming)']))

In [8]:
political_barriers /= sum(political_barriers)
political_barriers

AGO    0.002683
ALB    0.005715
ARG    0.006256
ARM    0.006595
AUS    0.005036
AUT    0.007003
AZE    0.002091
BDI    0.002412
BEL    0.007003
BEN    0.005100
BFA    0.005115
BGD    0.008034
BGR    0.006593
BIH    0.005961
BLR    0.003100
BLZ    0.007808
BOL    0.010757
BRA    0.006481
BTN    0.003773
BWA    0.007009
CAF    0.001617
CAN    0.005250
CHE    0.007028
CHL    0.006979
CHN    0.002145
         ...   
SYC    0.015298
SYR   -0.000232
TCD    0.002402
TGO    0.010441
THA    0.003947
TJK    0.001720
TKM    0.000761
TON    0.005300
TTO    0.007473
TUN    0.006613
TUR    0.006381
TUV    0.020064
TZA    0.009758
UGA    0.007655
UKR    0.006188
URY    0.006386
USA    0.005064
UZB    0.000598
VEN    0.003699
VNM    0.002633
VUT    0.008521
WSM    0.017174
YEM    0.002034
ZAF    0.005807
ZMB    0.007690
Length: 158, dtype: float64

### UN Migration History

In [54]:
un_pd = pd.read_excel(
        file_path(
            "UN_MigrantStockByOriginAndDestination_2015.xlsx"
        ),
        skiprows=15
    )

In [55]:
names = un_pd.iloc[:,1]
un_codes = un_pd.iloc[:,3]
names_to_un_codes = {names[i]: un_codes[i] for i in range(len(names))}

In [56]:
set(un_codes).difference(set(cc.iloc[:,1]).union(set(other_codes.iloc[:,1])))

{830,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 920,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 931,
 934,
 935,
 941,
 947,
 954,
 957,
 5500,
 5501}

In [36]:
un_pd.iloc[:,1:250]

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Total,Other North,Other South,Afghanistan,Albania,Algeria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,WORLD,NaN,900,NaN,243700236,2139539.0,7644005.0,4843117.0,1122910.0,1763771.0,...,346976.0,1991040.0,8583.0,606344.0,2558678.0,7843.0,91034.0,1012889.0,238121.0,856345.0
1,Developed regions,(b),901,NaN,140481955,539780.0,3520214.0,462239.0,1118223.0,1673604.0,...,161328.0,1500105.0,2604.0,485509.0,2232557.0,2.0,90.0,71411.0,72945.0,252569.0
2,Developing regions,(c),902,NaN,103218281,1599759.0,4123791.0,4380878.0,4687.0,90167.0,...,185648.0,490935.0,5979.0,120835.0,326121.0,7841.0,90944.0,941478.0,165176.0,603776.0
3,Least developed countries,(d),941,NaN,11951316,241805.0,1005567.0,0.0,0.0,10449.0,...,593.0,751.0,66.0,5198.0,75802.0,0.0,0.0,410.0,57538.0,82930.0
4,Less developed regions excluding least develop...,NaN,934,NaN,91266965,1357954.0,3118224.0,4380878.0,4687.0,79718.0,...,185055.0,490184.0,5913.0,115637.0,250319.0,7841.0,90944.0,941068.0,107638.0,520846.0
5,Sub-Saharan Africa,(e),947,NaN,18993986,328171.0,1551645.0,122.0,164.0,11433.0,...,117.0,0.0,0.0,305.0,164.0,0.0,0.0,494.0,164274.0,603208.0
6,Africa,NaN,903,NaN,20649557,350543.0,1586186.0,677.0,599.0,40194.0,...,117.0,502.0,0.0,343.0,164.0,0.0,90939.0,32482.0,164304.0,603246.0
7,Eastern Africa,NaN,910,NaN,6129113,102542.0,457137.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,410.0,87781.0,82861.0
8,Burundi,NaN,108,B R,286810,3242.0,20599.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Comoros,NaN,174,B,12555,597.0,754.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
#un_pd.iloc[:,1:250].columns.values

In [65]:
codes = []
for code in [names_to_un_codes[x] for x in un_pd.iloc[:,1:250].columns.values[7:239]]:
    if len(cc[cc['Numeric code'] == code]):
        codes.append(cc[cc['Numeric code'] == code].index[0])
    #elif len(other_codes[other_codes['Code'] == country]):
    #    codes.append(other_codes[other_codes['Code'] == country]["ISO"][0])
    else:
        print(code)

535
830
531
534
728
729
